# Импорт данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from catboost import Pool, CatBoostClassifier
import numpy as np
import re
from string import punctuation
from sklearn.metrics import roc_curve, auc

# Загрузка данных

In [3]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

# Чтение данных таблицы user_data
query = "SELECT * FROM user_data"
user_data = pd.read_sql(query, engine)

# Чтение данных таблицы post_text_df
query = "SELECT * FROM post_text_df"
post_text_df = pd.read_sql(query, engine)

# Чтение ограниченного количества данных таблицы feed_data
query = "SELECT * FROM feed_data LIMIT 1000000"
feed_data = pd.read_sql(query, engine)

# Переименование столбцов идентификаторов
user_data = user_data.rename(columns={'id': 'user_id'})
post_text_df = post_text_df.rename(columns={'id': 'post_id'})

# Объединение таблиц
data = feed_data.merge(user_data, on='user_id', how='left')
data = data.merge(post_text_df, on='post_id', how='left')

# Обработка временных меток

In [4]:
# Преобразование формата временных меток в объект datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Извлечение признаков из временных меток
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['hour_of_day'] = data['timestamp'].dt.hour

# Расчет времени с момента последнего действия для каждого пользователя
data = data.sort_values(['user_id', 'timestamp'])
data['time_since_last_action'] = data.groupby('user_id')['timestamp'].diff().dt.total_seconds()
data['time_since_last_action'].fillna(0, inplace=True)

# Удаление столбца временных меток
data = data.drop('timestamp', axis=1)

# Обучение модели CatBoost

## Train-test split

Этот код формирует выборку данных с заданными признаками, выбирая топ-k признаков с использованием взаимной информации, без утечки данных, временной метки, 'action' и 'text'. Затем данные разбиваются на обучающую и тестовую выборки с заданным отношением размеров, и рандомным состоянием генератора псевдослучайных чисел.

In [8]:
X = data.drop(['target', 'action', 'text'], axis=1)

In [9]:
X

,user_id,post_id,gender,age,country,city,exp_group,os,source,topic,day_of_week,hour_of_day,time_since_last_action
0,15409,3174,1,27,Russia,Sergiyev Posad,3,iOS,ads,covid,5,12,0.0
1,15409,1141,1,27,Russia,Sergiyev Posad,3,iOS,ads,politics,5,22,1247702.0
2,15409,5930,1,27,Russia,Sergiyev Posad,3,iOS,ads,movie,5,22,160.0
3,15409,1920,1,27,Russia,Sergiyev Posad,3,iOS,ads,tech,5,22,150.0
4,15409,1920,1,27,Russia,Sergiyev Posad,3,iOS,ads,tech,5,22,118.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,88048,1442,1,19,Russia,Tyumen,0,Android,ads,sport,0,13,31.0
999996,88048,3432,1,19,Russia,Tyumen,0,Android,ads,covid,0,13,164.0
999997,88048,7057,1,19,Russia,Tyumen,0,Android,ads,movie,0,13,28.0
999998,88048,5789,1,19,Russia,Tyumen,0,Android,ads,movie,0,13,59.0


In [10]:
# Выборка целевой переменной
y = data['target']

# Убедиться, что каждый столбец уникален
X = X.loc[:,~X.columns.duplicated()]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Целевой кодировщик с предварительным сглаживанием.
Источник: https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69

## Обучение модели на Precision@5 

Мы создаем группы данных на основе идентификатора пользователя 'user_id', чтобы иметь возможность проводить обучение с учетом группировки данных. Затем мы сортируем данные по группам и создаем объекты Pool для обучения и тестирования с колонкой 'group_id', которые затем будут использоваться для обучения модели и оценки ее производительности.

In [14]:
from catboost import CatBoostClassifier, Pool

# Замените названия столбцов на соответствующие вашим данным
categorical_columns = ['country', 'topic', 'city', 'gender', 'os', 'source']
cat_features = [X_train.columns.get_loc(col) for col in categorical_columns]

# Создание train_pool и test_pool с указанием категориальных столбцов
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(X_test, label=y_test, cat_features=cat_features)

precision_model = CatBoostClassifier(iterations=1000,
                                     learning_rate=0.1,
                                     depth=6,
                                     custom_metric='AUC',
                                     eval_metric='AUC',
                                     random_seed=42,
                                     verbose=100)

precision_model.fit(train_pool, eval_set=test_pool)


0:	test: 0.5647659	best: 0.5647659 (0)	total: 295ms	remaining: 4m 55s
100:	test: 0.6483719	best: 0.6483719 (100)	total: 11.5s	remaining: 1m 42s
200:	test: 0.6508937	best: 0.6509013 (198)	total: 24.1s	remaining: 1m 35s
300:	test: 0.6514159	best: 0.6514201 (299)	total: 37.4s	remaining: 1m 26s
400:	test: 0.6517193	best: 0.6517598 (388)	total: 50.9s	remaining: 1m 15s
500:	test: 0.6518894	best: 0.6518920 (499)	total: 1m 4s	remaining: 1m 3s
600:	test: 0.6519299	best: 0.6519716 (565)	total: 1m 17s	remaining: 51.6s
700:	test: 0.6519503	best: 0.6519716 (565)	total: 1m 31s	remaining: 39s
800:	test: 0.6518366	best: 0.6519716 (565)	total: 1m 45s	remaining: 26.1s
900:	test: 0.6518494	best: 0.6519716 (565)	total: 1m 58s	remaining: 13.1s
999:	test: 0.6517609	best: 0.6519716 (565)	total: 2m 12s	remaining: 0us

bestTest = 0.651971559
bestIteration = 565

Shrink model to first 566 iterations.


In [16]:
# Создание ID группы на основе столбца 'user_id'
unique_user_ids = X_train['user_id'].unique()
group_id_dict = {user_id: idx for idx, user_id in enumerate(unique_user_ids)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict)
X_test['group_id'] = X_test['user_id'].map(group_id_dict)

# Сортировка наборов данных для обучения и тестирования по 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index]

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

# Убедитесь, что категориальные переменные представлены в виде строк
categorical_columns = ['country', 'topic', 'city', 'gender', 'os', 'source']
X_train[categorical_columns] = X_train[categorical_columns].astype(str)
X_test[categorical_columns] = X_test[categorical_columns].astype(str)

# Получение индексов категориальных столбцов
cat_features = [X_train.drop(columns=['user_id']).columns.get_loc(col) for col in categorical_columns]

# Создание объектов Pool для обучающей и тестовой выборок с колонкой 'group_id' и категориальными признаками
train_pool = Pool(X_train.drop(columns=['user_id']), y_train, cat_features=cat_features, group_id=X_train['group_id'])
test_pool = Pool(X_test.drop(columns=['user_id']), y_test, cat_features=cat_features, group_id=X_test['group_id'])


In [17]:
# Обучение модели CatBoost с использованием метрики PrecisionAt:top=5
from catboost import CatBoostClassifier

precision_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='PrecisionAt:top=5',
                           eval_metric='PrecisionAt:top=5',
                           random_seed=42,
                           verbose=100)

precision_model.fit(train_pool, eval_set=test_pool)


0:	learn: 0.1050598	test: 0.1234591	best: 0.1234591 (0)	total: 147ms	remaining: 2m 27s
100:	learn: 0.1878565	test: 0.1780129	best: 0.1785649 (95)	total: 12.9s	remaining: 1m 54s
200:	learn: 0.2126035	test: 0.1796688	best: 0.1796688 (200)	total: 26.8s	remaining: 1m 46s
300:	learn: 0.2316467	test: 0.1801288	best: 0.1817847 (263)	total: 41s	remaining: 1m 35s
400:	learn: 0.2441582	test: 0.1808648	best: 0.1824287 (359)	total: 55.8s	remaining: 1m 23s
500:	learn: 0.2578657	test: 0.1793008	best: 0.1824287 (359)	total: 1m 11s	remaining: 1m 10s
600:	learn: 0.2691812	test: 0.1786569	best: 0.1824287 (359)	total: 1m 26s	remaining: 57.2s
700:	learn: 0.2840846	test: 0.1793008	best: 0.1824287 (359)	total: 1m 41s	remaining: 43.2s
800:	learn: 0.2942042	test: 0.1793928	best: 0.1824287 (359)	total: 1m 56s	remaining: 28.8s
900:	learn: 0.3038638	test: 0.1798528	best: 0.1824287 (359)	total: 2m 11s	remaining: 14.4s
999:	learn: 0.3126035	test: 0.1802208	best: 0.1824287 (359)	total: 2m 26s	remaining: 0us

bestTe

# Сохранение и загрузка модели CatBoost

In [18]:
precision_model.save_model('catboost_precision_model.cbm')

In [20]:
from catboost import CatBoostClassifier

# Загрузка сохраненной модели
loaded_model = CatBoostClassifier()
loaded_model.load_model('catboost_precision_model.cbm')

# Предсказание с использованием загруженной модели на тестовом наборе данных
predictions = loaded_model.predict(test_pool)

# Оценка загруженной модели на тестовом наборе данных
score = loaded_model.score(test_pool)
print("Accuracy:", score)

# Вычисление других метрик, если это необходимо
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.894
Precision: 0.7992360000000001
Recall: 0.894
F1-score: 0.8439662090813094


c:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Обучение модели на Recall@5 

In [21]:
recall_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='RecallAt:top=5',
                           eval_metric='RecallAt:top=5',
                           random_seed=42,
                           verbose=100)

recall_model.fit(train_pool, eval_set=test_pool)


0:	learn: 0.0188194	test: 0.0981062	best: 0.0981062 (0)	total: 151ms	remaining: 2m 30s
100:	learn: 0.0310298	test: 0.1329637	best: 0.1334783 (96)	total: 12.9s	remaining: 1m 54s
200:	learn: 0.0349209	test: 0.1345620	best: 0.1345857 (153)	total: 26.7s	remaining: 1m 46s
300:	learn: 0.0381617	test: 0.1345814	best: 0.1356039 (262)	total: 41s	remaining: 1m 35s
400:	learn: 0.0407466	test: 0.1357123	best: 0.1359474 (358)	total: 55.5s	remaining: 1m 22s
500:	learn: 0.0426862	test: 0.1348660	best: 0.1359474 (358)	total: 1m 10s	remaining: 1m 10s
600:	learn: 0.0447767	test: 0.1348362	best: 0.1359474 (358)	total: 1m 25s	remaining: 56.5s
700:	learn: 0.0470848	test: 0.1351843	best: 0.1359474 (358)	total: 1m 40s	remaining: 42.8s
800:	learn: 0.0484569	test: 0.1348453	best: 0.1361425 (709)	total: 1m 55s	remaining: 28.6s
900:	learn: 0.0503225	test: 0.1355986	best: 0.1361681 (899)	total: 2m 10s	remaining: 14.3s
999:	learn: 0.0514157	test: 0.1364315	best: 0.1370372 (953)	total: 2m 25s	remaining: 0us

bestTe

In [22]:
recall_model.save_model('catboost_recall_model.cbm')

## Обучение модели на MAP@5 

In [23]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='PFound:top=5',
                           eval_metric='PFound:top=5',
                           random_seed=42,
                           verbose=100)

model.fit(train_pool, eval_set=test_pool)


0:	test: 0.3469695	best: 0.3469695 (0)	total: 140ms	remaining: 2m 20s
100:	test: 0.4903148	best: 0.4929616 (88)	total: 11.9s	remaining: 1m 46s
200:	test: 0.4962268	best: 0.4962268 (200)	total: 25s	remaining: 1m 39s
300:	test: 0.4949184	best: 0.4980397 (215)	total: 38.5s	remaining: 1m 29s
400:	test: 0.4954035	best: 0.4980397 (215)	total: 52.2s	remaining: 1m 17s
500:	test: 0.4951972	best: 0.4980397 (215)	total: 1m 6s	remaining: 1m 6s
600:	test: 0.4989122	best: 0.4989122 (600)	total: 1m 20s	remaining: 53.2s
700:	test: 0.4973819	best: 0.4989122 (600)	total: 1m 34s	remaining: 40.2s
800:	test: 0.4986868	best: 0.5009043 (788)	total: 1m 48s	remaining: 26.9s
900:	test: 0.5007371	best: 0.5032109 (826)	total: 2m 1s	remaining: 13.4s
999:	test: 0.4992821	best: 0.5032109 (826)	total: 2m 15s	remaining: 0us

bestTest = 0.5032109274
bestIteration = 826

Shrink model to first 827 iterations.


In [24]:
model.save_model('catboost_MAP_model.cbm')

- bestTest = 0.4451217663 for MAP@5
- bestTest = 0.1603712149 for Recall@5
- bestTest = 0.1660869565 for Precision@5

# Сравнение моделей

In [25]:
from catboost import CatBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

models = {
    'recall': 'catboost_recall_model.cbm',
    'MAP': 'catboost_MAP_model.cbm',
    'precision': 'catboost_precision_model.cbm',
}

metrics = {}

for model_name, model_path in models.items():
    loaded_model = CatBoostClassifier()
    loaded_model.load_model(model_path)

    predictions = loaded_model.predict(test_pool)

    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
    }

# Вывод метрик для каждой модели
for model_name, model_metrics in metrics.items():
    print(f"{model_name} model:")
    for metric_name, metric_value in model_metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")
    print()


recall model:
  Precision: 0.8522
  Recall: 0.8940
  F1-score: 0.8440

MAP model:
  Precision: 0.8257
  Recall: 0.8940
  F1-score: 0.8440

precision model:
  Precision: 0.7992
  Recall: 0.8940
  F1-score: 0.8440



c:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Для выбора модели, которая будет оцениваться по Hitrate@5, нужно посмотреть на метрику PrecisionAt:top=5 для каждой модели. Чем выше PrecisionAt:top=5, тем лучше модель справляется с задачей рекомендации топ-5 элементов.

Из предоставленных результатов, мы видим следующую картину:

- recall model: Precision: 0.8522
- MAP model: Precision: 0.8257
- precision model: Precision: 0.7992

Исходя из этой информации, лучшей моделью для оценки по метрике Hitrate@5 будет модель recall, так как у нее наивысшая точность (Precision) среди всех моделей. Мы будем использовать эту модель для рекомендации топ-5 элементов в вашей задаче.